Lo primero se debe imporatr las librerias que vamosa utlizar para cargar losda tos a la abse de datos, que ene sta caso son pandas para el manejo de losd atos. 

In [56]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import pandas as pd
import psycopg2
from src.database.db_conection import connect_db
from src.database.database_create import create_database
from src.database.raw_candidates import create_raw_table




In [43]:
data = pd.read_csv('../data/candidates.csv', sep=";", encoding='utf-8')
data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")
print(data.head())

   first_name   last_name                      email application_date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   country  yoe  seniority                         technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   code_challenge_score  technical_interview_score  
0                     3                          3  
1                     2                         10  
2          

In [44]:
data.count()

first_name                   50000
last_name                    50000
email                        50000
application_date             50000
country                      50000
yoe                          50000
seniority                    50000
technology                   50000
code_challenge_score         50000
technical_interview_score    50000
dtype: int64

depues de leer la csv con los datos, ahora vamos a crear a la base de datos

In [45]:
# Llamar a la función para crear la base de datos
if __name__ == "__main__":
    print("Creando la base de datos...")
    create_database()  
    print("Base de datos creada exitosamente")

Creando la base de datos...
Base de datos 'candidates' creada exitosamente.
Base de datos creada exitosamente


depues creamos la tabla 


In [46]:
if __name__ == "__main__":
    print("Creando la tabla...")
    create_raw_table()
    print("Tabla creada exitosamente.")

Creando la tabla...
Tabla raw_candidates creada correctamente.
Tabla creada exitosamente.


Cargamos el CSV a la base de datos y tablas ya creadas.

In [ ]:
conn = connect_db(db_name="candidates")
cursor = conn.cursor()

In [ ]:

# Insertar datos en la tabla
for _, row in data.iterrows():
    cursor.execute(
        """
        INSERT INTO raw_candidates 
        (first_name, last_name, email, application_date, country, yoe, seniority, technology, 
        code_challenge_score, technical_interview_score) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            row["first_name"], row["last_name"], row["email"], row["application_date"],
            row["country"], row["yoe"], row["seniority"], row["technology"],
            row["code_challenge_score"], row["technical_interview_score"]
        )
    )

conn.commit()
print("Datos insertados correctamente en la base de datos.")




Datos insertados correctamente en la base de datos


In [57]:
# Leer datos con Pandas usando psycopg2 (obteniendo un DataFrame)
df = pd.read_sql_query("SELECT * FROM raw_candidates", conn)
print(df.head())


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21928\2455145920.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM raw_candidates", conn)


   id  first_name   last_name                      email application_date  \
0   1  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1   2      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2   3       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3   4        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4   5       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   country  yoe  seniority                         technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   code_challenge_score  technical_interview_score  
0                     3                          3  
1                     2                 

In [ ]:
# Cerrar conexión
cursor.close()
conn.close()